In [ ]:
from flask import request, abort, current_app as app
from flask import Flask
import re
import os
import time
import pickle
import numpy as np
import requests
from sklearn.preprocessing import Normalizer

line_seen = set()
response = 0
check = 0
ip_timelog = []
ip_ban_list = []
ip_whitelist = [('127.0.0.1')]
ipr = []
PLAYER_NAME = []
returned_data = []
newplayername = []
newplayerskills = []
DATAarray = []
userfixed = ""
osrsknn_predict = -1

tempnames = list()
tempgroups = []
tempind = 0

MAIN = "http://services.runescape.com/m=hiscore_oldschool/index_lite.ws?player="

osrsknn = pickle.load(open("OSRS_KNN_V1","rb"))
y_km = pickle.load(open("ykmfile","rb"))
PLAYER_TRAIN = pickle.load(open("traindata","rb"))
player_name = pickle.load(open("pnamefile","rb"))
PLAYER_IND = pickle.load(open("PIfile","rb"))

normal = Normalizer()
PLAYER_IND = np.reshape(PLAYER_IND,(-1,78))
PLAYER_IND = normal.fit_transform(PLAYER_IND)

for name in player_name:
    PLAYER_NAME.append(name.replace('\n',''))
    
##################################################################################################

app = Flask(__name__)

@app.before_request
def block_method(r = 0):
    ip = request.environ.get('REMOTE_ADDR')
    
    if ip in ipr:
        r = ipr.index(ip)
        check = 1
    else:
        ip_timelog.append([ip,time.time()])
        check = 0
        ipr.append(ip)
    
    last_time = ip_timelog[r][1]
    response = time.time()-last_time
    ip_timelog[r][1] = time.time()
    
    if check == 1:
        if response < 1:
            if ip not in ip_whitelist:
                if ip in ip_ban_list:
                    abort(403)
                else:
                    ip_ban_list.append(ip)
                    abort(403)
        else:
            if ip not in ip_whitelist:
                if ip in ip_ban_list:
                    abort(403)
    print("Response Time: ", response)
    print("Timelog: ", ip_timelog)
    print("IP REC: ", ipr)
    print("Banlist: ", ip_ban_list)
    print("Whitelist: ", ip_whitelist)

@app.route('/', methods =['POST'])
def post():
    if request.method == 'POST':
        print(request.data)
        datastr = str(request.data)
        dataclean = datastr[1:].replace('\\r', '').strip("'[]").replace(', ','\n')
        with open("INPUT.txt", "wt") as SERVERDATARAW:
            SERVERDATARAW.write(dataclean)
        SERVERDATARAW.close()
        SERVERDATARAW = open("TempINPUT.txt", "rt")
        with open("PLAYERGATHERDATA2.txt", "a") as SERVERDATA:
            for line in SERVERDATARAW:
                if len(line)<13:
                    L = re.findall('[a-zA-Z0-9_-] *', line)
                    line = ''.join(map(str, L))
                    if line not in line_seen:
                        SERVERDATA.write(line + '\n')
                        line_seen.add(line)
    return 
        
@app.route('/user/<user>', methods =['GET'])
def get(user):
    global osrsknn_predict
    userfixed = user.replace(' ','_')
    print(userfixed)
    if userfixed in PLAYER_NAME:
        ind = PLAYER_NAME.index(userfixed)
        osrsknn_predict = osrsknn.predict(PLAYER_TRAIN[ind].reshape(1,-1))
        player_predprob = osrsknn.predict_proba(PLAYER_TRAIN[ind].reshape(1,-1))
        print(osrsknn_predict)
        print(player_predprob)
        print(y_km[ind])
        print(PLAYER_NAME[ind])
        return str(getResponse(osrsknn_predict)).strip('[]')
    else:
        if userfixed in tempnames:
            tempind = tempnames.index(userfixed)
            print("Name and Group taken from Storage.")
            return str(tempgroups[tempind]).strip('[]')
        else:
            try:
                print("User not found. Currently evaluating user...")
                pulldata(userfixed)
                return str(getResponse(osrsknn_predict)).strip('[]')
            except: 
                print("CODE ERROR")
    return 
        
def pulldata(userfixed):
    global osrsknn_predict
    global tempnames
    url = MAIN+userfixed 
    response = requests.get(url) 
    data = response.text 
    try: 
        if data.find('404 - Page not found') != -1:
            osrsknn_predict = -1
        else:
            r = str.split(data) 
            DATAarray = [[float(n) for n in row.split(",")] for row in r]
            cleanup(DATAarray)
            tempnames.append(userfixed)
            print("Name list",tempnames)
    except: 
        print("INTERNAL ERROR")
    return

def cleanup(DATAarray):
    global normal
    global newplayerskills
    DATAcheck = np.asarray(DATAarray)
    for x in range(0,len(DATAcheck)):
        if (0<x<24):
            newplayerskills = np.append(newplayerskills,DATAcheck[x][2])
        if (24<x<80):
            newplayerskills = np.append(newplayerskills,DATAcheck[x][1])

    newplayerskills = np.asarray(newplayerskills.reshape(-1, 78))
    newplayerskills = normal.transform(newplayerskills)
    
    print(newplayerskills)
    print("DATA CLEANED FOR USER")
    osrsKNN(newplayerskills)
    
    newplayerskills = []
    return 

def osrsKNN(newplayerskills):
    global osrsknn_predict
    print("SCANNING...")
    osrsknn_predict = osrsknn.predict(newplayerskills.reshape(1, -1))
    player_predprob = osrsknn.predict_proba(newplayerskills)
    print("Group: "+str(osrsknn_predict))
    print("Player Grouping Data: "+str(player_predprob))
    print("PLAYER SUCCESSFULLY SCANNED")
    tempgroups.append(osrsknn_predict)
    print(tempgroups)
    return 

def getResponse(osrsknn_predict):
    return osrsknn_predict

if __name__ == '__main__':
    app.run(port="8000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Mar/2021 23:47:04] "GET / HTTP/1.1" 405 -


Response Time:  0.0
Timelog:  [['127.0.0.1', 1615265224.0972733]]
IP REC:  ['127.0.0.1']
Banlist:  []
Whitelist:  ['127.0.0.1']


127.0.0.1 - - [08/Mar/2021 23:47:18] "GET / HTTP/1.1" 405 -


Response Time:  14.40144419670105
Timelog:  [['127.0.0.1', 1615265238.4987175]]
IP REC:  ['127.0.0.1']
Banlist:  []
Whitelist:  ['127.0.0.1']
Response Time:  2.996966600418091
Timelog:  [['127.0.0.1', 1615265241.4956841]]
IP REC:  ['127.0.0.1']
Banlist:  []
Whitelist:  ['127.0.0.1']
Settled
User not found. Currently evaluating user...


c:\users\thear\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
127.0.0.1 - - [08/Mar/2021 23:47:23] "GET /user/Settled HTTP/1.1" 200 -


[[ 3.08771054e-01  2.31349013e-01  2.46711479e-01  4.64038673e-01
   4.68994954e-01  1.30875018e-01  1.86924127e-01  1.36467707e-01
   1.37182044e-01  1.45735397e-01  1.30587534e-01  1.31587322e-01
   1.31475462e-01  1.33666248e-01  1.31117301e-01  1.34185176e-01
   1.32940020e-01  1.70219140e-01  1.32624546e-01  1.49489018e-01
   1.32515271e-01  1.35081278e-01  1.30858759e-01  1.60286572e-07
   2.00358215e-08  3.30591055e-06 -1.00179108e-08  4.80859717e-07
   6.51164200e-07  1.68300901e-06  4.70841806e-07  2.00358215e-08
   3.03542696e-05 -1.00179108e-08 -1.00179108e-08 -1.00179108e-08
   1.37946631e-05 -1.00179108e-08 -1.00179108e-08  1.12300780e-05
   3.80680609e-06 -1.00179108e-08 -1.00179108e-08 -1.00179108e-08
   1.23220302e-06  5.00895538e-06 -1.00179108e-08  1.77317021e-06
   2.08372544e-06  1.81324185e-06 -1.00179108e-08  2.73488964e-06
   1.60386751e-05 -1.00179108e-08 -1.00179108e-08 -1.00179108e-08
  -1.00179108e-08 -1.00179108e-08 -1.00179108e-08  3.42612548e-06
  -1.00179

127.0.0.1 - - [08/Mar/2021 23:48:11] "GET /user/Settled HTTP/1.1" 200 -


Response Time:  50.34554576873779
Timelog:  [['127.0.0.1', 1615265291.84123]]
IP REC:  ['127.0.0.1']
Banlist:  []
Whitelist:  ['127.0.0.1']
Settled
Name and Group taken from Storage.
